In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree-ensenble').getOrCreate()

In [3]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [4]:
data = spark.read.format('libsvm').load('../data/sample_libsvm_data.txt')

In [5]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [6]:
train_data, test_data = data.randomSplit([.7, .3], seed=42)

In [7]:
dtc = DecisionTreeClassifier(maxDepth=5, seed=42)
rfc = RandomForestClassifier(maxDepth=5, numTrees=10, seed=42)
gbc = GBTClassifier(maxDepth=5, maxIter=100, stepSize=0.1, seed=42)

In [8]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbc_model = gbc.fit(train_data)

In [9]:
dtc_result = dtc_model.transform(test_data)
rfc_result = rfc_model.transform(test_data)
gbc_result = gbc_model.transform(test_data)

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
evaluator = MulticlassClassificationEvaluator()

for n, result in zip(['DTC', 'RFC', 'GBT'], [dtc_result, rfc_result, gbc_result]):
    metric = 'accuracy'
    m = evaluator.setMetricName(metric).evaluate(result)
    print(f'{n} -> {metric.upper()}: {m}')

DTC -> ACCURACY: 0.9428571428571428
RFC -> ACCURACY: 1.0
GBT -> ACCURACY: 0.9428571428571428


In [12]:
college = spark.read.csv('../data/College.csv', inferSchema=True, header=True)

In [13]:
college.printSchema()
college.describe()
college.show(10)

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|E

In [14]:
college.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [15]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

In [16]:
assembler = VectorAssembler(
    inputCols=[
        'Apps',
        'Accept',
        'Enroll',
        'Top10perc',
        'Top25perc',
        'F_Undergrad',
        'P_Undergrad',
        'Outstate',
        'Room_Board',
        'Books',
        'Personal',
        'PhD',
        'Terminal',
        'S_F_Ratio',
        'perc_alumni',
        'Expend',
        'Grad_Rate'
    ], outputCol='features'
)

private_indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

pp = Pipeline(stages=[assembler, private_indexer])

In [17]:
college_train, college_test = college.randomSplit([.7, .3], seed=42)

college_pipeline = pp.fit(college_train)

college_train = college_pipeline.transform(college_train)
college_test = college_pipeline.transform(college_test)

feature_col = 'features'
label_col = 'PrivateIndex'

dtc = DecisionTreeClassifier(featuresCol=feature_col, labelCol=label_col, seed=42)
rfc = RandomForestClassifier(featuresCol=feature_col, labelCol=label_col, seed=42)
gbc = GBTClassifier(featuresCol=feature_col, labelCol=label_col, seed=42)

dtc_model = dtc.fit(college_test)
rfc_model = rfc.fit(college_test)
gbc_model = gbc.fit(college_test)

In [18]:
evaluator = MulticlassClassificationEvaluator(labelCol='PrivateIndex')

for n, model in zip(['DTC', 'RFC', 'GBT'], [dtc_model, rfc_model, gbc_model]):
    result = model.transform(college_test)
    for metric in ['accuracy', 'weightedPrecision', 'weightedRecall']:
        m = evaluator.setMetricName(metric).evaluate(result)
        print(f'{n} -> {metric.upper()}: {m}')
    print('-' * 50)

DTC -> ACCURACY: 0.995
DTC -> WEIGHTEDPRECISION: 0.9950312499999999
DTC -> WEIGHTEDRECALL: 0.9950000000000001
--------------------------------------------------
RFC -> ACCURACY: 0.99
RFC -> WEIGHTEDPRECISION: 0.99
RFC -> WEIGHTEDRECALL: 0.99
--------------------------------------------------
GBT -> ACCURACY: 1.0
GBT -> WEIGHTEDPRECISION: 1.0
GBT -> WEIGHTEDRECALL: 1.0
--------------------------------------------------
